In [1]:
from gurobipy import *
import numpy as np

In [33]:
#Function to print results of an optimization problem
def printResult(m):
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('\nMinimal cost %g \n' % m.objVal)

##L-shaped algorithm
#remaining things to address
# 1. m2 == 0, n2 == 0, n1 == 0
def LShapedAlgo(c, q, A, b, W, T, h, p):
    n1 = len(c)
    m1 = len(b)
    n2 = len(q)
    m2 = W.shape[0]
    K = len(p)
    
    #Initialization
    optimal = False
    r = 0
    s = 0
    nu = 0
    
    #Create master problem
    master = Model('master problem')
    master.setParam('OutPutFlag', False)
    x = master.addMVar(n1, lb=0, name='x')
    master.setObjective(c @ x, GRB.MINIMIZE)
    if m1 > 0:
        master.addConstr(A @ x == b)
    theta_val = float("-inf")
    
    #L-shaped iterations
    while(not optimal):
        infease = False
        nu += 1
        master.optimize()
        if s >= 1:
            theta_val = theta.x
        print('\nmaster problem %d solved' % nu)
        print('x = ')
        print(master.x)
        print('theta = ')
        print(theta_val)

        x_cur = x.x
        #Feasibility cuts
        for k in range(K):
            hk = h[k]
            Tk = T[k]
            fcut = Model('feasibilityCut')
            fcut.setParam('OutPutFlag', False)
            y = fcut.addMVar(n2, lb=0, name='y')
            vp = fcut.addMVar(m2, lb=0, name='vplus')
            vm = fcut.addMVar(m2, lb=0, name='vminus')
            e = np.ones(m2)
            fcut.setObjective(e @ vp + e @ vm)
            fcut.addConstr(W @ y + vp - vm == hk - np.dot(Tk, x_cur))
            fcut.optimize()
            if fcut.objVal > 0:
                r += 1
                pi = np.array(fcut.pi)
                D = np.dot(pi,T)
                d = np.dot(pi,hk)
                print('feasibility cut %d created with\n D =' % r)
                print(D)
                print('d=%f'%d)
                master.addConstr(D @ x >= d)
                infease = True
                break
        if infease:
            continue

        ##Optimality cuts
        E = np.zeros(n1)
        e = 0
        for k in range(K):
            hk = h[k]
            Tk = T[k]
            ocut = Model('optimalityCut')
            ocut.setParam('OutPutFlag', False)
            y = ocut.addMVar(n2,lb=0, name='y')
            ocut.setObjective(q @ y, GRB.MINIMIZE)
            ocut.addConstr(W @ y == hk - np.dot(Tk,x_cur))
            ocut.optimize()            
            pi = np.array(ocut.pi) 
            E += p[k]*np.dot(pi,Tk)
            e += p[k]*np.dot(pi,hk)
        if theta_val >= e - np.dot(E,x_cur):
            optimal = True
            print('theta: %f >= e-E\'X = %f ' % (theta_val, e - np.dot(E,x_cur)))
            print('optimality reached')
            printResult(master)
        else:
            s += 1
            if s == 1:
                theta = master.addMVar(1,lb=-GRB.INFINITY, name='theta')
                master.setObjective(c @ x + theta, GRB.MINIMIZE)
            master.addConstr(E @ x + theta >= e)
            print('optimality cut %d created with \n E = ' % s)
            print(E)
            print('e = %f' % e)

In [35]:
##Test L-shape
c = np.array([3,2])
q = np.array([-15,-12,0,0,0,0,0,0])
p = np.array([0.25,0.25,0.25,0.25])

T = []
for i in range(len(p)):
    T.append(np.array([[-1,0],
                  [0,-1],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,0]]))
T = np.array(T)
W = np.array([[3,2,1,0,0,0,0,0],
             [2,5,0,1,0,0,0,0],
             [1,0,0,0,-1,0,0,0], 
             [1,0,0,0,0,1,0,0],
             [0,1,0,0,0,0,-1,0],
             [0,1,0,0,0,0,0,1]])

h = []
psy = [[4,4],[4,8],[6,4],[6,8]]
for psyk in psy:
    h.append([0,0,psyk[0]*0.8, psyk[0], psyk[1]*0.8, psyk[1]])
h = np.array(h)

LShapedAlgo(c, q, [], [], W, T, h, p)


master problem 1 solved
x = 
[0.0, 0.0]
theta = 
-inf
feasibility cut 1 created with
 D =
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
d=6.400000

master problem 2 solved
x = 
[0.0, 6.4]
theta = 
-inf
feasibility cut 2 created with
 D =
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
d=6.400000

master problem 3 solved
x = 
[6.4, 0.0]
theta = 
-inf
feasibility cut 3 created with
 D =
[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
d=6.400000

master problem 4 solved
x = 
[6.4, 6.4]
theta = 
-inf
feasibility cut 4 created with
 D =
[[0.27272727 0.09090909]
 [0.27272727 0.09090909]
 [0.27272727 0.09090909]
 [0.27272727 0.09090909]]
d=6.400000

master problem 5 solved
x = 
[21.333333333333336, 6.4]
theta = 
-inf
feasibility cut 5 created with
 D =
[[0.  0.2]
 [0.  0.2]
 [0.  0.2]
 [0.  0.2]]
d=4.480000

master problem 6 solved
x = 
[16.0, 22.400000000000002]
theta = 
-inf
feasibility cut 6 created with
 D =
[[0.  0.2]
 [0.  0.2]
 [0.  0.2]
 [0.  0.2]]
d=7.680000

master problem 7 solved
x = 
[10.666666666666666, 38.4

In [25]:
Tk

NameError: name 'Tk' is not defined

In [10]:
##Define parameters
c = np.array([3,2])
q = np.array([-15,-12,0,0,0,0,0,0])
T = np.array([[-1,0],
              [0,-1],
              [0,0],
              [0,0],
              [0,0],
              [0,0]])
W = np.array(
    [[3,2,1,0,0,0,0,0],
     [2,5,0,1,0,0,0,0],
     [1,0,0,0,-1,0,0,0], 
     [1,0,0,0,0,1,0,0],
     [0,1,0,0,0,0,-1,0],
     [0,1,0,0,0,0,0,1]])
lbx = [0,0]
lby = [0,0,0,0,0,0,0,0]
psy = [[4,4],[4,8],[6,4],[6,8]]
p = [0.25, 0.25, 0.25, 0.25]
h = []
for psyk in psy:
    h.append([0,0,psyk[0]*0.8, psyk[0], psyk[1]*0.8, psyk[1]])

nx = len(c)
ny = len(q)
nyc = len(W)
K = len(psy)

optimal = False
r = 0
s = 0
nu = 0
#create master problem
master = Model('master problem')
master.setParam('OutPutFlag', False)

x = master.addMVar(nx, lb=lbx, name='x')
master.setObjective(c @ x, GRB.MINIMIZE)
theta_val = float("-inf")

In [11]:
##L-shaped iterations
while(not optimal):
    infease = False
    nu += 1
    master.optimize()
    if s >= 1:
        theta_val = theta.x
    print('master problem %d solved' % nu)
    print('x = ')
    print(master.x)
    print('theta = ')
    print(theta_val)
    
    x_cur = x.x
    #Feasibility cuts
    for k in range(K):
        hk = h[k]
        fcut = Model('feasibilityCut')
        fcut.setParam('OutPutFlag', False)
        y = fcut.addMVar(ny, lb=0, name='y')
        vp = fcut.addMVar(nyc, lb=0, name='vplus')
        vm = fcut.addMVar(nyc, lb=0, name='vminus')
        e = np.ones(nyc)
        fcut.setObjective(e @ vp + e @ vm)
        fcut.addConstr(W @ y + vp - vm == hk - np.dot(T, x_cur))
        fcut.optimize()
        if fcut.objVal > 0:
            r += 1
            pi = np.array(fcut.pi)
            D = np.dot(pi,T)
            d = np.dot(pi,hk)
            print('feasibility cut %d created with\n D =' % r)
            print(D)
            print('d=%f'%d)
            master.addConstr(D @ x >= d)
            infease = True
            break
    if infease:
        continue
    
    ##Optimality cuts
    E = np.zeros(nx)
    e = 0
    for k in range(K):
        ocut = Model('optimalityCut')
        ocut.setParam('OutPutFlag', False)
        hk = h[k]
        y = ocut.addMVar(ny,lb=0, name='y')
        ocut.setObjective(q @ y, GRB.MINIMIZE)
        ocut.addConstrs((W[i] @ y == hk[i] - np.dot(T[i],x_cur) \
                         for i in range(nyc)))
        ocut.optimize()            
        pi = np.array(ocut.pi) 
        E += p[k]*np.dot(pi,T)
        e += p[k]*np.dot(pi,hk)
    if theta_val >= e - np.dot(E,x_cur):
        optimal = True
        print('theta: %.2f >= e-E\'X = %f ' % (theta_val, e - np.dot(E,x_cur)))
        print('optimality reached')
        printResult(master)
    else:
        s += 1
        if s == 1:
            theta = master.addMVar(1,lb=-GRB.INFINITY, name='theta')
            master.setObjective(c @ x + theta, GRB.MINIMIZE)
        master.addConstr(E @ x + theta >= e)
        print('optimality cut %d created with \n E = ' % s)
        print(E)
        print('e = %f' % e)

master problem 1 solved
x = 
[0.0, 0.0]
theta = 
-inf
feasibility cut 1 created with
 D =
[1. 1.]
d=6.400000
master problem 2 solved
x = 
[0.0, 6.4]
theta = 
-inf
feasibility cut 2 created with
 D =
[1. 0.]
d=6.400000
master problem 3 solved
x = 
[6.4, 0.0]
theta = 
-inf
feasibility cut 3 created with
 D =
[0. 1.]
d=6.400000
master problem 4 solved
x = 
[6.4, 6.4]
theta = 
-inf
feasibility cut 4 created with
 D =
[0.27272727 0.09090909]
d=6.400000
master problem 5 solved
x = 
[21.333333333333336, 6.4]
theta = 
-inf
feasibility cut 5 created with
 D =
[0.  0.2]
d=4.480000
master problem 6 solved
x = 
[16.0, 22.400000000000002]
theta = 
-inf
feasibility cut 6 created with
 D =
[0.  0.2]
d=7.680000
master problem 7 solved
x = 
[10.666666666666666, 38.4]
theta = 
-inf
feasibility cut 7 created with
 D =
[0.33333333 0.        ]
d=5.333333
master problem 8 solved
x = 
[16.000000000000004, 38.4]
theta = 
-inf
feasibility cut 8 created with
 D =
[0.33333333 0.        ]
d=7.466667
master proble

In [30]:
a,b = W.shape

In [32]:
b

8

In [252]:
print(nu)

1
